### Extract Transform Load (ETL)

ETL is one of the first things which needs to be done in a data science project. The nature of this task highly depends on the type of data source. Whether it is relational or unstructured, enterprise data or internet data, persistent data or streaming data. This heavily influences the choice of architecture. Therefore, you must document your choice and thinking process in the Architectural Decision Document (ADD).

This task involves – as the name implies – accessing the data source, transforming it in a way it can be easily worked with and finally make it available to downstream analytics processes – either real-time streaming or batch ones.

In case of operational relational data, de-normalization usually needs to take place, for unstructured data, some feature extraction might already be appropriate and for real-time data, windows are usually created.

Please create an ETL process, document it and save this deliverable according to the naming convention of the process model.

In [ ]:
!pip install tensorflow
!pip install python-mnist
!pip install Pillow
!pip install pyspark

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import image
import tensorflow as tf
import seaborn as sns
from mnist import MNIST
import numpy as np
import PIL
from PIL import Image
import os
import matplotlib.image as mping
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from numpy import asarray
import pandas as pd

In [ ]:
tf.__version__

In [ ]:
PIL.__version__

## First dataset

#### 60,000 images and 10 classes, each image is 28 x 28 or represented by a 794 element array

In [ ]:
# need to download and gunzip each of the train and test images and labels

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz?raw=True
!mv train-images-idx3-ubyte.gz?raw=True train-images-idx3-ubyte.gz
!gunzip train-images-idx3-ubyte.gz
!ls -lahr train-images-idx3-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz?raw=True
!mv train-labels-idx1-ubyte.gz?raw=True train-labels-idx1-ubyte.gz
!gunzip train-labels-idx1-ubyte.gz
!ls -lahr train-labels-idx1-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz?raw=True
!mv t10k-images-idx3-ubyte.gz?raw=True t10k-images-idx3-ubyte.gz
!gunzip t10k-images-idx3-ubyte.gz
!ls -lahr t10k-images-idx3-ubyte

!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz?raw=True
!mv t10k-labels-idx1-ubyte.gz?raw=True t10k-labels-idx1-ubyte.gz
!gunzip t10k-labels-idx1-ubyte.gz
!ls -lahr t10k-labels-idx1-ubyte

In [ ]:
# download the classmap for the first dataset, add the phonetics, and display it
# the classmap shows the classes that each written kuzushiji can be classified into
# and we add the phonetic sound that the characters makes when spoken

url = "http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist_classmap.csv"
df_classmap = pd.read_csv(url)
phonetic = ['o','ki','su','tsu','na','ha','ma','ya','re','wo']
df_classmap['phonetic'] = phonetic
df_classmap

In [ ]:
# create a directory and move these files into it
# then show what is in the directory

!mkdir kmnistdata
!cp t10k-images-idx3-ubyte kmnistdata/t10k-images-idx3-ubyte
!cp t10k-labels-idx1-ubyte kmnistdata/t10k-labels-idx1-ubyte
!cp train-images-idx3-ubyte kmnistdata/train-images-idx3-ubyte
!cp train-labels-idx1-ubyte kmnistdata/train-labels-idx1-ubyte
!ls -al kmnistdata

In [ ]:
# load the data into arrays

data = MNIST('kmnistdata')
train_images, train_labels = data.load_training()
test_images, test_labels = data.load_testing()

In [ ]:
print(train_labels[0])
print(train_images[0])

In [ ]:
type(train_labels)

In [ ]:
type(train_images)

In [ ]:
# the data needs to be transformed into numpy arrays for use  in the model

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
type(train_images[0])

In [ ]:
type(train_images)

In [ ]:
# the data as downloaded has two dimensions
# there are 60000 images, each image is represented
# in one dimension as 784 numbers

# (see the notebook kuzushiji.data_exp for displayed images)

train_images.shape

In [ ]:
# the train_images are now a numpy array of numbers between 0 and 255
print(train_images[0])

In [ ]:
# we need to normalize the numpy arrays so that each number in the numpy
# array is between 0 and 1:

train_images = train_images / 255
test_images = test_images / 255

In [ ]:
train_images[0]

## Second dataset

#### 232,365 images, 49 classes, each image is 28 x 28 or represented by a 794 element array

In [ ]:
# download the train and test labels and images, 
# which are in .npz format 

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz?raw=True
!mv k49-train-imgs.npz?raw=True k49-train-imgs.npz
!ls -lahr k49-train-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz?raw=True
!mv k49-train-labels.npz?raw=True k49-train-labels.npz
!ls -lahr k49-train-labels.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz?raw=True
!mv k49-test-imgs.npz?raw=True k49-test-imgs.npz
!ls -lahr k49-test-imgs.npz

!wget http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz?raw=True
!mv k49-test-labels.npz?raw=True k49-test-labels.npz
!ls -lahr k49-test-labels.npz

In [ ]:
# we need to define a load function in order to extract the data
# from the .npz format, and then use  it to extract:

def load(f):
    return np.load(f)['arr_0']

k49_train_images = load('k49-train-imgs.npz')
k49_train_labels = load('k49-train-labels.npz')
k49_test_images = load('k49-test-imgs.npz')
k49_test_labels = load('k49-test-labels.npz')

In [ ]:
# next we download the classmap for this dataset
# and add the phonetic, and display it:

url = "http://codh.rois.ac.jp/kmnist/dataset/k49/k49_classmap.csv"
df_k49_classmap = pd.read_csv(url)

k49_phonetic = ['a', 'i',  'u',  'e',  'o',\
                'ka','ki', 'ku', 'ke', 'ko',\
                'sa','shi','su', 'se', 'so',\
                'ta','chi','tsu','te', 'to',\
                'na','ni', 'nu', 'ne', 'no',\
                'ha','hi', 'fu', 'he', 'ho',\
                'ma','mi', 'mu', 'me', 'mo',\
                'ya','yu', 'yo',\
                'ra','ri', 'ru', 're', 'ro',\
                'wa','wi', 'we ','wo', 'n','iteration_mark']
print(len(k49_phonetic))
df_k49_classmap['phonetic'] = k49_phonetic
df_k49_classmap

In [ ]:
# the data is already a numpy array
# since we pulled it out of the .nps formatted structure

type(k49_train_images)

In [ ]:
# need to check the dimensionality of the data:

k49_train_images.shape

In [ ]:
k49_train_labels.shape

In [ ]:
k49_test_images.shape

In [ ]:
# this means we need to flatten the data
# to two dimensions

k49_train_images = np.reshape(k49_train_images, (232365, 784))
k49_test_images = np.reshape(k49_test_images, (38547, 784))

In [ ]:
k49_train_images.shape

In [ ]:
k49_test_images.shape

In [ ]:
# the k49_train_images are now a numpy array of numbers between 0 and 255
print(k49_train_images[0])

In [ ]:
# we need to normalize the numpy arrays so that each number in the numpy
# array is between 0 and 1:

k49_train_images = k49_train_images / 255
k49_test_images = k49_test_images / 255

In [ ]:
print(k49_train_images[0])

In [ ]:
# this is the format we need for the machine learning model

## The Third Dataset 

#### includes Kanji characters, this data has 3832 different classes and consists of 140,426 images, each image is 64 X 64 pixels
#### This dataset is not as processed as the other two.  It is just a bunch of png images in a directory inside an archive file.

In [ ]:
# we download the archive:

!wget http://codh.rois.ac.jp/kmnist/dataset/kkanji/kkanji.tar?raw=True
!mv kkanji.tar?raw=True kkanji.tar
!ls -lahr kkanji.tar

# list the contents of the archive
# limit output to the first 70 files

!tar -tf kkanji.tar | head -70

In [ ]:
# extract the archive:
!tar -xf kkanji.tar

# check whats in the current working directory:
!ls -al

# check the extracted png files in the newly extracted directory
# List out the first 50 folder names in the directory kkanji2
# the folder names are also the codepoint of each of the characters:

!ls kkanji2 | head -50

In [ ]:
# put the codepoints in a file
# let the first line of the file be name
# of the column in the dataframe we are creating
!echo codepoint > codepoints.csv
!ls kkanji2 >> codepoints.csv
!cat codepoints.csv | head -50

In [ ]:
# make sure the file codepoints.csv is in the current directory:

!ls -al

In [ ]:
# verify the file's path:

print (os.path.abspath("codepoints.csv"))

In [ ]:
# read all the lines of the file into pandas dataframe
# including the column header which is already in the file
# display new dataframe
# here we can confirm that the data has 3832 classes

df_kanji_classmap = pd.read_csv("codepoints.csv")
df_kanji_classmap

In [ ]:
# we'll need to read the images in to transform them
#read the first listed image in the first folder and display it
img = mping.imread('kkanji2/U+5B87/72d56fcb33d10fe0.png')
plt.imshow(img)
plt.show()
# note that this image in only "first" as listed from the tar arcive above
# it is not "first" in terms of the codepoint listing we created directly above

In [ ]:
# create a pandas dataframe that contains the codepoint for each image, 
# and its full path in the os and display that dataframe

data = []
dir = os.path.realpath('/home/wsuser/work/kkanji2')
for r, d, f in os.walk(dir):
    for file in f:
        if ".png" in file:
            data.append((r.split('/')[-1],os.path.join(r,file)))
df_kanji2 = pd.DataFrame(data, columns=['codepoint', 'image_file_path']).sort_values(by=['codepoint'], ignore_index = True)

In [ ]:
# display the new dataframe
# we will change the display width of the pandas 
# dataframe to be able to see the entire path

pd.set_option('max_colwidth', 1000)
df_kanji2

In [ ]:
# show the os path of the first image in the first folder
# as listed in the dataframe
# note that the dataframe list the images grouped by their codepoints
# and the codepoints are sorted by their ASCII values

df_kanji2['image_file_path'][0]

In [ ]:
# read the image using Pillow and show some information about it:

pimage = Image.open(df_kanji2['image_file_path'][0])
print(pimage.format)
print(pimage.size)
print(pimage.mode)

In [ ]:
# read the image using matplotlib and show it
img = image.imread(df_kanji2['image_file_path'][0])
print(img.dtype)
plt.imshow(img)

In [ ]:
# convert to a numpy array and verify we created a numpy array and it shape
img_nparray = asarray(img)
print(type(img_nparray))
print(img_nparray.shape)

In [ ]:
# show the values of the array
# notice the array is already normalized
img_nparray

In [ ]:
img_nparray.shape

In [ ]:
# add a blank column to the dataframe with column name 'np_array'

df_kanji2['np_array'] = ""
df_kanji2

In [ ]:
type(df_kanji2['np_array'])


In [ ]:
type(df_kanji2['np_array'][0])

In [ ]:
df_kanji2['np_array'].shape

In [ ]:
df_kanji2['np_array'][0].shape

In [ ]:
# add a column containing and numpy array of the image indicated in the path in image_file_path
# and at the same time, flatten each image from a 64 x 64 numpy array to a single
# dimension 4096 element long numpy array
df_kanji2['np_array'] = df_kanji2['image_file_path'].apply(lambda x: np.asarray(Image.open(x))).apply(lambda y: np.reshape(y,(4096,)))



In [ ]:
df_kanji2

In [ ]:
type(df_kanji2['np_array'])

In [ ]:
type(df_kanji2['np_array'][0])

In [ ]:
df_kanji2['np_array'].shape

In [ ]:
df_kanji2['np_array'][0].shape

In [ ]:
# let's normalize the numpy arrays:
df_kanji2['np_array'] = df_kanji2['np_array'] / 255